In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

2023-03-21 01:38:16.921256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 01:38:17.347310: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-21 01:38:18.553417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 01:38:18.553609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
#define our CNN model
classifier = Sequential()

#add convolutional layer
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

#add pooling layer
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#flatten the layers
classifier.add(Flatten())

#add dense layer
classifier.add(Dense(units = 128, activation = 'relu'))

#add output layer
classifier.add(Dense(units = 1, activation = 'sigmoid'))

#compile the model
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

2023-03-21 01:38:20.554862: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-21 01:38:20.554915: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ArchLinux): /proc/driver/nvidia/version does not exist
2023-03-21 01:38:20.556663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 01:38:20.725433: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15745024 exceeds 10% of free system memory.
2023-03-21 01:38:20.733589: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15745024 exceeds 10% of free system memory.
2023-03-21 01:38

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

NameError: name 'X' is not defined

In [ ]:
train_directory = 'Dataset_split/train'
test_directory = 'Dataset_split/test'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_directory, target_size = (64, 64), batch_size = 32, class_mode = 'binary')

In [ ]:
test_set = test_datagen.flow_from_directory(test_directory, target_size = (64, 64), batch_size = 32, class_mode = 'binary')

In [ ]:
classifier.fit_generator(training_set, steps_per_epoch = 8000, epochs = 25, validation_data = test_set, validation_steps = 2000)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)